In [54]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Configura el controlador
chromedriver_path = '/Users/danielebelmiro/Downloads/chromedriver-mac-arm64/chromedriver'  # Reemplaza con la ruta de tu chromedriver
service = Service(chromedriver_path)
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# Abre la página
url = 'https://tienda.mercadona.es/categories'
driver.get(url)

# Desplazamiento hasta el final de la página
TIEMPO_ESPERA_DESPLAZAMIENTO = 2
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(TIEMPO_ESPERA_DESPLAZAMIENTO)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Espera explícita para asegurar la carga de las categorías principales
WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, 'category-menu__header'))
)

# Extrae el HTML de la página cargada
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Encuentra las categorías principales y sus subcategorías
categorias = []
categorias_principales = soup.find_all('span', class_='category-menu__header')

# Verificar si se encontraron las categorías principales
print(f'Número de categorías principales encontradas: {len(categorias_principales)}')

for categoria in categorias_principales:
    categoria_nombre = categoria.find('label', class_='subhead1-r')

    if categoria_nombre:
        categoria_nombre_texto = categoria_nombre.text.strip()
        print(f'Procesando categoría: {categoria_nombre_texto}')  # Debug

        # Encuentra el elemento <ul> correspondiente para subcategorías
        lista_subcategorias = soup.find_all('li', class_="subhead1-r category-item")
                
        if lista_subcategorias:
            # Itera sobre cada subcategoría
            for subcat in lista_subcategorias:                
                
                subcat_nombre = subcat.find('button', class_='category-item__link')
                if subcat_nombre:
                    subcat_nombre_texto = subcat_nombre.text.strip()
                    categorias.append({
                        "categoria": categoria_nombre_texto,
                        "subcategoria": subcat_nombre_texto
                    })
                    print(f'  Agregando subcategoría: {subcat_nombre_texto}')  # Debug
        else:
            print(f'  No se encontraron subcategorías para: {categoria_nombre_texto}')  # Debug
            categorias.append({
                "categoria": categoria_nombre_texto,
                "subcategoria": ""
            })

# Crea la tabla de categorías con subcategorías
tabla_categorias = pd.DataFrame(categorias)
print(tabla_categorias)

# Cierra el controlador
driver.quit()


Número de categorías principales encontradas: 26
Procesando categoría: Aceite, especias y salsas
  Agregando subcategoría: Especias
  Agregando subcategoría: Mayonesa, ketchup y mostaza
  Agregando subcategoría: Otras salsas
Procesando categoría: Agua y refrescos
  Agregando subcategoría: Especias
  Agregando subcategoría: Mayonesa, ketchup y mostaza
  Agregando subcategoría: Otras salsas
Procesando categoría: Aperitivos
  Agregando subcategoría: Especias
  Agregando subcategoría: Mayonesa, ketchup y mostaza
  Agregando subcategoría: Otras salsas
Procesando categoría: Arroz, legumbres y pasta
  Agregando subcategoría: Especias
  Agregando subcategoría: Mayonesa, ketchup y mostaza
  Agregando subcategoría: Otras salsas
Procesando categoría: Azúcar, caramelos y chocolate
  Agregando subcategoría: Especias
  Agregando subcategoría: Mayonesa, ketchup y mostaza
  Agregando subcategoría: Otras salsas
Procesando categoría: Bebé
  Agregando subcategoría: Especias
  Agregando subcategoría: Mayo

In [52]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Configura o driver
chromedriver_path = '/Users/danielebelmiro/Downloads/chromedriver-mac-arm64/chromedriver'  # Substitua pelo caminho do seu chromedriver
service = Service(chromedriver_path)
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# Abre a página
url = 'https://tienda.mercadona.es/categories'
driver.get(url)

# Fecha pop-up se aparecer
try:
    # Aguarda o elemento do pop-up
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME, 'modal__close-button'))
    ).click()
    print("Pop-up fechado.")
except:
    print("Nenhum pop-up encontrado.")

# Rolagem para o final da página para carregar o conteúdo dinâmico
SCROLL_PAUSE_TIME = 2
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Espera explícita para garantir o carregamento das categorias principais
WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, 'category-menu__header'))
)

# Encontra as categorias principais usando Selenium
categorias = []
categorias_principais = driver.find_elements(By.CLASS_NAME, 'category-menu__header')

print(f'Número de categorias principais encontradas: {len(categorias_principais)}')

# Itera sobre cada categoria principal
for categoria in categorias_principais:
    # Extrai o nome da categoria principal
    categoria_nome = categoria.text.strip()

    # Faz scroll para o elemento e clica via JavaScript
    driver.execute_script("arguments[0].scrollIntoView(true);", categoria)
    driver.execute_script("arguments[0].click();", categoria)
    time.sleep(1)  # Espera um segundo para garantir que as subcategorias carreguem

    # Recarrega o HTML da página após o clique para capturar as subcategorias
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Verifica se encontra a categoria correspondente usando o novo método com `string`
    categoria_container = soup.find('span', string=categoria_nome)
    if categoria_container:
        categoria_container = categoria_container.find_parent('div')  # Obtém o contêiner pai

        # Verifica e busca subcategorias dentro do contêiner
        if categoria_container:
            subcategorias_lista = categoria_container.find_all('li', class_='category-item')

            # Adiciona as subcategorias se forem encontradas
            if subcategorias_lista:
                for subcat in subcategorias_lista:
                    subcat_nome = subcat.get_text(strip=True)
                    categorias.append({
                        "categoria": categoria_nome,
                        "subcategoria": subcat_nome
                    })
                    print(f'  Adicionando subcategoria: {subcat_nome}')  # Debug
            else:
                # Adiciona a categoria sem subcategoria se nenhuma subcategoria for encontrada
                print(f'  Nenhuma subcategoria encontrada para: {categoria_nome}')  # Debug
                categorias.append({
                    "categoria": categoria_nome,
                    "subcategoria": ""
                })
    else:
        print(f'  Contêiner da categoria não encontrado para: {categoria_nome}')  # Debug

# Cria a tabela de categorias com subcategorias
tabela_categorias = pd.DataFrame(categorias)
print(tabela_categorias)

# Encerra o driver
driver.quit()


Nenhum pop-up encontrado.
Número de categorias principais encontradas: 26
  Contêiner da categoria não encontrado para: Aceite, especias y salsas
  Contêiner da categoria não encontrado para: Agua y refrescos
  Contêiner da categoria não encontrado para: Aperitivos
  Contêiner da categoria não encontrado para: Arroz, legumbres y pasta
  Contêiner da categoria não encontrado para: Azúcar, caramelos y chocolate
  Contêiner da categoria não encontrado para: Bebé
  Contêiner da categoria não encontrado para: Bodega
  Contêiner da categoria não encontrado para: Cacao, café e infusiones
  Contêiner da categoria não encontrado para: Carne
  Contêiner da categoria não encontrado para: Cereales y galletas
  Contêiner da categoria não encontrado para: Charcutería y quesos
  Contêiner da categoria não encontrado para: Congelados
  Contêiner da categoria não encontrado para: Conservas, caldos y cremas
  Contêiner da categoria não encontrado para: Cuidado del cabello
  Contêiner da categoria não en

In [53]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

# URL da página
url = 'https://supermercado.eroski.es/'

# Faz a solicitação HTTP para obter o conteúdo HTML da página
response = requests.get(url)
response.raise_for_status()  # Lança um erro se a resposta não for bem-sucedida

# Cria o objeto BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Encontrar as categorias principais
categorias = []
categorias_principales = soup.find_all('span', class_="label_menu") 

# Verifique se as categorias principais foram encontradas
print(f'Número de categorias principais encontradas: {len(categorias_principales)}')

# Itera sobre cada categoria principal encontrada
for categoria in categorias_principales:
    # Extrai o nome da categoria principal
    categoria_nombre = categoria.get_text(strip=True)

    if categoria_nombre:
        print(f'Procesando categoría: {categoria_nombre}')  # Debug

        # Tenta encontrar a lista de subcategorias associadas ao contêiner correto
        categoria_container = categoria.find_parent('div')  # Contêiner da categoria principal
        if categoria_container:
            lista_subcategorias = categoria_container.find_all('li', class_='nav-item-2059988')

            # Adiciona subcategorias, se encontradas
            if lista_subcategorias:
                for subcat in lista_subcategorias:
                    subcat_nombre = subcat.get_text(strip=True)
                    categorias.append({
                        "categoria": categoria_nombre,
                        "subcategoria": subcat_nombre
                    })
                    print(f'  Agregando subcategoría: {subcat_nombre}')  # Debug
            else:
                # Adiciona uma entrada vazia se nenhuma subcategoria for encontrada
                print(f'  No se encontraron subcategorías para: {categoria_nombre}')  # Debug
                categorias.append({
                    "categoria": categoria_nombre,
                    "subcategoria": ""
                })

# Cria a tabela de categorias com subcategorias
tabela_categorias = pd.DataFrame(categorias)
print(tabela_categorias)


Número de categorias principais encontradas: 15
Procesando categoría: Alimentación
  Agregando subcategoría: Aceite, vinagre, sal, harina y pan ralladoVer todo Aceite, vinagre, sal, harina y pan ralladoAceite de girasolAceite de oliva intensoAceite de oliva suaveAceite de oliva virgenOtros aceitesHarinaPan ralladoPastillas caldoSalVinagres especialesVinagre
  Agregando subcategoría: Ver todo Aceite, vinagre, sal, harina y pan rallado
Procesando categoría: Frescos
  Agregando subcategoría: Aceite, vinagre, sal, harina y pan ralladoVer todo Aceite, vinagre, sal, harina y pan ralladoAceite de girasolAceite de oliva intensoAceite de oliva suaveAceite de oliva virgenOtros aceitesHarinaPan ralladoPastillas caldoSalVinagres especialesVinagre
  Agregando subcategoría: Ver todo Aceite, vinagre, sal, harina y pan rallado
Procesando categoría: Dulces y desayuno
  Agregando subcategoría: Aceite, vinagre, sal, harina y pan ralladoVer todo Aceite, vinagre, sal, harina y pan ralladoAceite de girasolA

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [21]:
url = 'https://tienda.mercadona.es/categories/'
headers = {
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, como Gecko) Chrome/130.0.0.0 Safari/537.36',
}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')


In [22]:
categorias = []
for categoria in soup.find_all('label', class_='subhead1-r'):
    categorias.append({
        "categoria" : categoria.text.strip()
    })


In [23]:
tabla_categorias = pd.DataFrame(categorias)
print(tabla_categorias)


Empty DataFrame
Columns: []
Index: []


In [2]:
respuesta_json =  ""
respuesta =  ""

In [4]:
category = '121'
url =  f"https://tienda.mercadona.es/categories/{category}"
    
response = requests.get(url)
response


<Response [200]>

In [7]:

url =  "https://tienda.mercadona.es/categories/"
    
response = requests.get(url)
response


<Response [200]>

In [8]:
response.content

b'<!doctype html><html lang="es"><head><meta charset="utf-8"><meta http-equiv="X-UA-Compatible" content="IE=edge"/><link rel="manifest" href="/manifest.json"><link rel="shortcut icon" href="/favicon.ico"><meta name="viewport" content="width=device-width,initial-scale=1,shrink-to-fit=no"><meta name="theme-color" content="#000000"><meta name="google" content="notranslate"><meta name="twitter:card" content="summary_large_image"><meta name="twitter:site" content="@Mercadona"><meta name="twitter:creator" content="@Mercadona"><meta name="google-site-verification" content="AiAA_avfXacCRnY2G3ZE91SUVhDaT-QiraT_Y0y8Bv4"/><script async src="https://www.googletagmanager.com/gtag/js?id=UA-119340431-4"></script><script>function gtag(){dataLayer.push(arguments)}window.dataLayer=window.dataLayer||[],gtag("js",new Date)</script><meta property="og:image" content="https://tienda.mercadona.es/web-display-img.png" data-react-helmet="true"><meta property="og:url" content="https://tienda.mercadona.es" data-r

In [9]:
response.headers # Response headers (as a Python dictionary)

{'Content-Type': 'text/html', 'Content-Length': '1507', 'Content-Encoding': 'gzip', 'Expires': 'Wed, 30 Oct 2024 13:47:54 GMT', 'Cache-Control': 'max-age=1800, public', 'X-Cache-Status': 'HIT', 'X-SRE-header': 'location_index', 'X-Frame-Options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1', 'Content-Security-Policy': "frame-ancestors 'self'", 'Referrer-Policy': 'origin-when-cross-origin', 'Strict-Transport-Security': 'max-age=86400', 'Date': 'Wed, 30 Oct 2024 13:17:54 GMT', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding'}

In [10]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.content, "html.parser")

In [11]:
print(soup.prettify())  # This formats the HTML in a readable way

<!DOCTYPE html>
<html lang="es">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible">
   <link href="/manifest.json" rel="manifest"/>
   <link href="/favicon.ico" rel="shortcut icon"/>
   <meta content="width=device-width,initial-scale=1,shrink-to-fit=no" name="viewport"/>
   <meta content="#000000" name="theme-color"/>
   <meta content="notranslate" name="google"/>
   <meta content="summary_large_image" name="twitter:card"/>
   <meta content="@Mercadona" name="twitter:site"/>
   <meta content="@Mercadona" name="twitter:creator"/>
   <meta content="AiAA_avfXacCRnY2G3ZE91SUVhDaT-QiraT_Y0y8Bv4" name="google-site-verification">
    <script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-119340431-4">
    </script>
    <script>
     function gtag(){dataLayer.push(arguments)}window.dataLayer=window.dataLayer||[],gtag("js",new Date)
    </script>
    <meta content="https://tienda.mercadona.es/web-display-img.png" data-react-helmet="true" prop

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import csv

# Configure Selenium
driver = webdriver.Chrome()
driver.get('https://tienda.mercadona.es/categories/77')

# Wait for the page to load completely
import time
time.sleep(5)  # Adjust as necessary

# Get HTML from the loaded page
html = driver.page_source

# Use BeautifulSoup to analyze HTML
soup = BeautifulSoup(html, 'html.parser')
# Update classes here
egg_names = soup.find_all('h4', class_='subhead1-r product-cell__description-name')
egg_sizes = soup.find_all('div', class_='product-format product-format__size--cell')
egg_prices = soup.find_all('p', class_='product-price__unit-price subhead1-b')

# Check if data were fetched
print(f"Eggs: {len(egg_names)}, Prices: {len(egg_prices)}, Sizes: {len(egg_sizes)}")

# Close browser
driver.quit()

# Save data in CSV file
if egg_names and egg_sizes and egg_prices:
    with open('eggs.csv', 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Egg', 'Size', 'Price'])
        for egg_name, egg_size, egg_price in zip(egg_names, egg_sizes, egg_prices):
            writer.writerow([egg_name.text.strip(), egg_size.text.strip(), egg_price.text.strip()])
           
    print("Data saved in eggs.csv")
else:
    print("No product or price found.")


In [7]:
categorias= []
for categoria in response_json["menu_analytics"].keys():
    for subcat in response_json["menu_analytics"][categoria]["children"].keys():
        if len(response_json["menu_analytics"][categoria]["children"][subcat]["path"].split("/")) == 5:
            categorias.append({
                "categoria" : response_json["menu_analytics"][categoria]["children"][subcat]["path"].split("/")[1],
                "subcategoria" : response_json["menu_analytics"][categoria]["children"][subcat]["path"].split("/")[2],
                "id" : response_json["menu_analytics"][categoria]["children"][subcat]["path"].split("/")[4]
            })
        else:
            continue


In [ ]:
categorias

[{'categoria': 'charcuteria-y-quesos',
  'subcategoria': 'jamon-cocido-lacon-fiambres-y-mortadela',
  'id': 'L2001'},
 {'categoria': 'charcuteria-y-quesos',
  'subcategoria': 'jamon-curado-y-paleta',
  'id': 'L2004'},
 {'categoria': 'charcuteria-y-quesos',
  'subcategoria': 'lomo-chorizo-fuet-salchichon',
  'id': 'L2005'},
 {'categoria': 'charcuteria-y-quesos',
  'subcategoria': 'queso-curado-semicurado-y-tierno',
  'id': 'L2007'},
 {'categoria': 'charcuteria-y-quesos',
  'subcategoria': 'queso-fresco',
  'id': 'L2008'},
 {'categoria': 'charcuteria-y-quesos',
  'subcategoria': 'queso-azul-y-roquefort',
  'id': 'L2009'},
 {'categoria': 'charcuteria-y-quesos',
  'subcategoria': 'quesos-fundidos-y-cremas',
  'id': 'L2010'},
 {'categoria': 'charcuteria-y-quesos',
  'subcategoria': 'quesos-internacionales',
  'id': 'L2011'},
 {'categoria': 'charcuteria-y-quesos',
  'subcategoria': 'foie-pate-y-sobrasada',
  'id': 'L2012'},
 {'categoria': 'charcuteria-y-quesos',
  'subcategoria': 'salchichas

In [9]:
tabla_categorias = pd.DataFrame(categorias)
list(tabla_categorias["subcategoria"])

['jamon-cocido-lacon-fiambres-y-mortadela',
 'jamon-curado-y-paleta',
 'lomo-chorizo-fuet-salchichon',
 'queso-curado-semicurado-y-tierno',
 'queso-fresco',
 'queso-azul-y-roquefort',
 'quesos-fundidos-y-cremas',
 'quesos-internacionales',
 'foie-pate-y-sobrasada',
 'salchichas',
 'vacuno',
 'cerdo',
 'pavo',
 'conejo',
 'hamburguesas-y-carne-picada',
 'pollo',
 'pescados',
 'ahumados-salazones-y-preparados',
 'sucedaneo-de-angulas-y-surimi',
 'mariscos',
 'ajos-cebollas-y-puerros',
 'tomates-pimientos-y-pepinos',
 'judias-brocolis-y-coliflores',
 'lechuga-escarolas-y-endivias',
 'patatas-y-zanahorias',
 'setas-y-champinones',
 'verduras-y-ensaladas-preparadas',
 'otras-verduras',
 'calabacin-calabaza-y-berenjena',
 'manzanas',
 'platanos',
 'peras',
 'uvas',
 'limones-y-pomelos',
 'frutas-del-bosque',
 'frutas-tropicales',
 'frutas-de-temporada',
 'frutas-deshidratadas',
 'naranjas-y-mandarinas',
 'arroz',
 'quinoa-y-couscous',
 'pastas',
 'alubias',
 'garbanzos',
 'lentejas',
 'aceit

In [10]:
tabla_categorias.head(58)

,categoria,subcategoria,id
0,charcuteria-y-quesos,jamon-cocido-lacon-fiambres-y-mortadela,L2001
1,charcuteria-y-quesos,jamon-curado-y-paleta,L2004
2,charcuteria-y-quesos,lomo-chorizo-fuet-salchichon,L2005
3,charcuteria-y-quesos,queso-curado-semicurado-y-tierno,L2007
4,charcuteria-y-quesos,queso-fresco,L2008
5,charcuteria-y-quesos,queso-azul-y-roquefort,L2009
6,charcuteria-y-quesos,quesos-fundidos-y-cremas,L2010
7,charcuteria-y-quesos,quesos-internacionales,L2011
8,charcuteria-y-quesos,foie-pate-y-sobrasada,L2012
9,charcuteria-y-quesos,salchichas,L2206


In [11]:
indice = tabla_categorias[tabla_categorias["subcategoria"] == "huevos"].index
indice

Index([54], dtype='int64')

In [12]:
numero = indice[0]
int(numero)

54

In [13]:
#funcion para tabla

In [14]:
respuesta_json =  ""
respuesta =  ""
lista_productos = {}
lista_producto_elegido = []
categoria = ""
subcategoria = ""
idL = 0
numero = 0
product = ""
cat = ""
subcat = ""


In [15]:
tabla_categorias[tabla_categorias["subcategoria"] == "bodega"].index

Index([186], dtype='int64')

In [16]:
numero = int(indice[0])
numero

54

In [17]:
def producto_elegido(producto_a_analizar):
    for linea in tabla_categorias["subcategoria"]:
        if producto_a_analizar in linea:
            indice = tabla_categorias[tabla_categorias["subcategoria"] == f"{linea}"].index
            numero = int(indice[0])
            lista_producto_elegido = [tabla_categorias["categoria"][numero], tabla_categorias["subcategoria"][numero], tabla_categorias["id"][numero]]
            print(lista_producto_elegido)
            print(type(lista_producto_elegido))
            print(lista_producto_elegido[0])
            cat = lista_producto_elegido[0]
            subcat = lista_producto_elegido[1]
            idL = lista_producto_elegido[2]
            return cat, subcat, idL
    return None, None, None


In [18]:
def obtener_html(cat, subcat, idL):
    url = f"https://www.dia.es/api/v1/plp-insight/initial_analytics/{cat}/{subcat}/c/{idL}?filters=categories:{idL}:es&locale=es&navigation={idL}&page=1&seo="
    

    payload = {}
    headers = {
    'accept': 'application/json',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
    'Cookie': 'session_id=1248a3ed-bcf3-4470-a94a-0b0117288549'
    }

    respuesta = requests.request("GET", url, headers=headers, data=payload)
    respuesta_json = respuesta.json()
    return respuesta_json


In [19]:
def obtener_tabla(respuesta_json):
    
    for id, daigual in respuesta_json["page_product_analytics"].items():
        for articulo in respuesta_json["page_product_analytics"][id]:
            lista_productos[respuesta_json["page_product_analytics"][id]["item_name"]] = respuesta_json["page_product_analytics"][id]["price"]

    tabla = pd.DataFrame(list(lista_productos.items()), columns=['articulo', 'precio'])
    tabla = tabla.sort_values("precio")
    tabla = tabla.reset_index(drop=True)
    print(tabla)

In [20]:
contador = 0

In [21]:

def accion_total(contador_1=0):
    global contador
    producto_a_analizar = input(f"Introduce el producto: ")
    cat,subcat,idL = producto_elegido(producto_a_analizar)
    if cat:
        respuesta_json = obtener_html(cat, subcat, idL)
        obtener_tabla(respuesta_json)
    else: 
        print("EL PRODUCTO NO EXISTE")
        contador = contador_1 + 1
        if contador == 3:
            pass
        else:
            accion_total(contador)

In [22]:
accion_total()

['verduras', 'tomates-pimientos-y-pepinos', 'L2023']
<class 'list'>
verduras
                                    articulo  precio
0                   pepino holandes 1 unidad    0.89
1           pimiento rojo unidad 390 g aprox    1.17
2          pimiento verde granel 500 g aprox    1.25
3           tomate cherry pera bandeja 250 g    1.29
4           pimiento semipicante bolsa 200 g    1.29
5                   pepino granel 1 kg aprox    1.69
6                 tomate canario malla 750 g    1.79
7             pimientos tricolor bolsa 500 g    1.89
8                tomate cherry bandeja 400 g    1.89
9          tomate ensalada granel 1 kg aprox    1.89
10                  tomate bio bandeja 500 g    1.99
11        pimiento sweet palermo bolsa 250 g    1.99
12  tomate cherry pera lobello bandeja 200 g    1.99
13        tomate maravilla bolsa 600 g aprox    2.14
14            tomate pera granel 900 g aprox    2.24
15               tomate murice bandeja 400 g    2.29
16          tomate bar

In [24]:
producto_a_analizar = input(f"Introduce el producto: ")
for linea in tabla_categorias["subcategoria"]:
    if producto_a_analizar in linea:
        indice = tabla_categorias[tabla_categorias["subcategoria"] == f"{linea}"].index
        numero = int(indice[0])
        lista_producto_elegido = [tabla_categorias["categoria"][numero], tabla_categorias["subcategoria"][numero], tabla_categorias["id"][numero]]
        print(lista_producto_elegido)

KeyboardInterrupt: Interrupted by user

In [ ]:
obtener_tabla(lista1)

# PRODUCTO 1 - HUEVOS

In [25]:
category_name = 'leche-huevos-y-mantequilla'
subcategory_name = 'huevos'
category = 'L2055'
url = f"https://www.dia.es/api/v1/plp-insight/initial_analytics/{category_name}/{subcategory_name}/c/{category}?filters=categories:{category}:es&locale=es&navigation={category}&page=1&seo="
url2 = f"https://www.dia.es/api/v1/plp-insight/initial_analytics/{category_name}/{subcategory_name}/c/{category}?filters=categories:{category}:es&locale=es&navigation={category}&page=1&seo="

payload = {}
headers = {
  'accept': 'application/json',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
  'Cookie': 'session_id=1248a3ed-bcf3-4470-a94a-0b0117288549'
}

respuesta = requests.request("GET", url, headers=headers, data=payload)



In [26]:
respuesta

<Response [200]>

In [27]:
respuesta_json = respuesta.json()

In [28]:
respuesta_json

{'filters_analytics': {'brand_description': {'Dia': {'click_parameter': 'dia',
    'filter_type': 'marca'},
   'Grupo de lucas': {'click_parameter': 'grupo de lucas',
    'filter_type': 'marca'},
   'Ovonatur': {'click_parameter': 'ovonatur', 'filter_type': 'marca'}},
  'caffeine_free': {},
  'dia_brand': {'true': {'click_parameter': 'true'}},
  'gluten_free': {'SI': {'click_parameter': 'si',
    'filter_type': 'sin gluten'}},
  'l1_category_description': {'Leche, huevos y mantequilla': {'click_parameter': 'leche huevos y mantequilla'}},
  'l2_category_description': {'Huevos': {'click_parameter': 'huevos'}},
  'nuts_free': {},
  'salt_free': {},
  'size': {'L': {'click_parameter': 'l', 'filter_type': 'formato'},
   'M': {'click_parameter': 'm', 'filter_type': 'formato'},
   'XL': {'click_parameter': 'xl', 'filter_type': 'formato'}}},
 'footer_analytics': {'cs_1': {'analytics_id': 'atencion al cliente',
   'link': '/l/atencion-al-cliente'},
  'inf_1': {'analytics_id': 'entrega rapida'},

In [29]:
lista_productos = {}
for id in respuesta_json["page_product_analytics"].keys():
    for articulo in respuesta_json["page_product_analytics"][id]:
        lista_productos[respuesta_json["page_product_analytics"][id]["item_name"]] = respuesta_json["page_product_analytics"][id]["price"]
lista_productos

{'huevos frescos maiz categoria a clase ml grupo de lucas caja 12 unidades': 2.91,
 'huevos frescos categoria a clase l dia bandeja 12 unidades': 2.31,
 'huevos frescos categoria a clase m dia bandeja 12 unidades': 2.06,
 'claras de huevo pasteurizadas ovonatur botella 500 ml': 1.95,
 'huevos frescos categoria a clase m dia bandeja 24 unidades': 3.88,
 'huevos frescos de gallinas criadas en suelo categoria a clase ml dia bandeja 12 unidades': 2.5,
 'huevos camperos categoria a clase ml dia bandeja 10 unidades': 2.54,
 'huevos ecologicos categoria a clase ml dia bandeja 6 unidades': 2.54,
 'huevos frescos categoria a clase xl dia bandeja 6 unidades': 1.72}

In [30]:
tabla = pd.DataFrame(list(lista_productos.items()), columns=['articulo', 'precio'])

tabla

,articulo,precio
0,huevos frescos maiz categoria a clase ml grupo...,2.91
1,huevos frescos categoria a clase l dia bandeja...,2.31
2,huevos frescos categoria a clase m dia bandeja...,2.06
3,claras de huevo pasteurizadas ovonatur botella...,1.95
4,huevos frescos categoria a clase m dia bandeja...,3.88
5,huevos frescos de gallinas criadas en suelo ca...,2.50
6,huevos camperos categoria a clase ml dia bande...,2.54
7,huevos ecologicos categoria a clase ml dia ban...,2.54
8,huevos frescos categoria a clase xl dia bandej...,1.72


In [31]:
tabla = tabla.sort_values("precio")
tabla 

,articulo,precio
8,huevos frescos categoria a clase xl dia bandej...,1.72
3,claras de huevo pasteurizadas ovonatur botella...,1.95
2,huevos frescos categoria a clase m dia bandeja...,2.06
1,huevos frescos categoria a clase l dia bandeja...,2.31
5,huevos frescos de gallinas criadas en suelo ca...,2.50
7,huevos ecologicos categoria a clase ml dia ban...,2.54
6,huevos camperos categoria a clase ml dia bande...,2.54
0,huevos frescos maiz categoria a clase ml grupo...,2.91
4,huevos frescos categoria a clase m dia bandeja...,3.88


In [32]:
tabla['unidades'] = tabla['articulo']
tabla

,articulo,precio,unidades
8,huevos frescos categoria a clase xl dia bandej...,1.72,huevos frescos categoria a clase xl dia bandej...
3,claras de huevo pasteurizadas ovonatur botella...,1.95,claras de huevo pasteurizadas ovonatur botella...
2,huevos frescos categoria a clase m dia bandeja...,2.06,huevos frescos categoria a clase m dia bandeja...
1,huevos frescos categoria a clase l dia bandeja...,2.31,huevos frescos categoria a clase l dia bandeja...
5,huevos frescos de gallinas criadas en suelo ca...,2.50,huevos frescos de gallinas criadas en suelo ca...
7,huevos ecologicos categoria a clase ml dia ban...,2.54,huevos ecologicos categoria a clase ml dia ban...
6,huevos camperos categoria a clase ml dia bande...,2.54,huevos camperos categoria a clase ml dia bande...
0,huevos frescos maiz categoria a clase ml grupo...,2.91,huevos frescos maiz categoria a clase ml grupo...
4,huevos frescos categoria a clase m dia bandeja...,3.88,huevos frescos categoria a clase m dia bandeja...


In [33]:
tabla['unidades'] = tabla['unidades'].replace(r'(?i)12', 12, regex=True)
tabla['unidades'] = tabla['unidades'].replace(r'(?i)24', 24, regex=True)
tabla['unidades'] = tabla['unidades'].replace(r'(?i)10', 10, regex=True)
tabla['unidades'] = tabla['unidades'].replace(r'(?i)500', 1, regex=True)
tabla['unidades'] = tabla['unidades'].replace(r'(?i)6', 6, regex=True)

tabla


C:\Users\Javi\AppData\Local\Temp\ipykernel_19620\264393299.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  tabla['unidades'] = tabla['unidades'].replace(r'(?i)6', 6, regex=True)


,articulo,precio,unidades
8,huevos frescos categoria a clase xl dia bandej...,1.72,6
3,claras de huevo pasteurizadas ovonatur botella...,1.95,1
2,huevos frescos categoria a clase m dia bandeja...,2.06,12
1,huevos frescos categoria a clase l dia bandeja...,2.31,12
5,huevos frescos de gallinas criadas en suelo ca...,2.50,12
7,huevos ecologicos categoria a clase ml dia ban...,2.54,6
6,huevos camperos categoria a clase ml dia bande...,2.54,10
0,huevos frescos maiz categoria a clase ml grupo...,2.91,12
4,huevos frescos categoria a clase m dia bandeja...,3.88,24


In [34]:
tabla["precio"] = tabla["precio"].astype(float)



In [35]:
tabla['precio_unidad'] = (tabla["precio"] / tabla['unidades'])

tabla

,articulo,precio,unidades,precio_unidad
8,huevos frescos categoria a clase xl dia bandej...,1.72,6,0.286667
3,claras de huevo pasteurizadas ovonatur botella...,1.95,1,1.950000
2,huevos frescos categoria a clase m dia bandeja...,2.06,12,0.171667
1,huevos frescos categoria a clase l dia bandeja...,2.31,12,0.192500
5,huevos frescos de gallinas criadas en suelo ca...,2.50,12,0.208333
7,huevos ecologicos categoria a clase ml dia ban...,2.54,6,0.423333
6,huevos camperos categoria a clase ml dia bande...,2.54,10,0.254000
0,huevos frescos maiz categoria a clase ml grupo...,2.91,12,0.242500
4,huevos frescos categoria a clase m dia bandeja...,3.88,24,0.161667


In [36]:
for id in respuesta_json["page_product_analytics"].keys():
    for articulo in respuesta_json["page_product_analytics"][id]:
        lista_productos[respuesta_json["page_product_analytics"][id]["item_name"]] = respuesta_json["page_product_analytics"][id]["price"]

In [37]:
tabla = tabla.sort_values("precio_unidad")
tabla

,articulo,precio,unidades,precio_unidad
4,huevos frescos categoria a clase m dia bandeja...,3.88,24,0.161667
2,huevos frescos categoria a clase m dia bandeja...,2.06,12,0.171667
1,huevos frescos categoria a clase l dia bandeja...,2.31,12,0.192500
5,huevos frescos de gallinas criadas en suelo ca...,2.50,12,0.208333
0,huevos frescos maiz categoria a clase ml grupo...,2.91,12,0.242500
6,huevos camperos categoria a clase ml dia bande...,2.54,10,0.254000
8,huevos frescos categoria a clase xl dia bandej...,1.72,6,0.286667
7,huevos ecologicos categoria a clase ml dia ban...,2.54,6,0.423333
3,claras de huevo pasteurizadas ovonatur botella...,1.95,1,1.950000


In [38]:
tabla = tabla.reset_index(drop=True)
tabla

,articulo,precio,unidades,precio_unidad
0,huevos frescos categoria a clase m dia bandeja...,3.88,24,0.161667
1,huevos frescos categoria a clase m dia bandeja...,2.06,12,0.171667
2,huevos frescos categoria a clase l dia bandeja...,2.31,12,0.192500
3,huevos frescos de gallinas criadas en suelo ca...,2.50,12,0.208333
4,huevos frescos maiz categoria a clase ml grupo...,2.91,12,0.242500
5,huevos camperos categoria a clase ml dia bande...,2.54,10,0.254000
6,huevos frescos categoria a clase xl dia bandej...,1.72,6,0.286667
7,huevos ecologicos categoria a clase ml dia ban...,2.54,6,0.423333
8,claras de huevo pasteurizadas ovonatur botella...,1.95,1,1.950000


In [39]:
tabla["articulo"][0]

'huevos frescos categoria a clase m dia bandeja 24 unidades'

In [40]:
print(f"""
    Supermercado: DIA
    Categoria: Huevos
    El producto mas barato es: {tabla["articulo"][0]}
    Tiene unidades: {tabla["unidades"][0]}
    Su precio por unidad es: {tabla["precio_unidad"][0]}
    El precio del producto es: {tabla["precio"][0]}
    """)


    Supermercado: DIA
    Categoria: Huevos
    El producto mas barato es: huevos frescos categoria a clase m dia bandeja 24 unidades
    Tiene unidades: 24
    Su precio por unidad es: 0.16166666666666665
    El precio del producto es: 3.88
    


# PRODUCTO 2 - BOLSAS DE BASURA

In [41]:
category_name = 'limpieza-y-hogar'
subcategory_name = 'bolsas-de-basura'
category = 'L2160'
url = f"https://www.dia.es/api/v1/plp-insight/initial_analytics/{category_name}/{subcategory_name}/c/{category}?filters=categories:{category}:es&locale=es&navigation={category}&page=1&seo="

payload = {}
headers = {
  'accept': 'application/json',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
  'Cookie': 'session_id=1248a3ed-bcf3-4470-a94a-0b0117288549'
}

respuesta = requests.request("GET", url, headers=headers, data=payload)



In [42]:
respuesta.content

b'{"filters_analytics":{"brand_description":{"Super Paco de Dia":{"click_parameter":"super paco de dia","filter_type":"marca"}},"dia_brand":{"true":{"click_parameter":"true"}},"gluten_free":{},"l1_category_description":{"Limpieza y hogar":{"click_parameter":"limpieza y hogar"}},"l2_category_description":{"Bolsas de basura":{"click_parameter":"bolsas de basura"}},"lactose_free":{},"nuts_free":{}},"footer_analytics":{"cs_1":{"analytics_id":"atencion al cliente","link":"/l/atencion-al-cliente"},"inf_1":{"analytics_id":"entrega rapida"},"inf_2":{"analytics_id":"envio"},"inf_3":{"analytics_id":"localizador tienda","link":"https://www.dia.es/tiendas/buscador-tiendas-folletos"},"inf_4":{"analytics_id":"folletos","link":"https://www.dia.es/folleto"},"inf_5":{"analytics_id":"tarjeta club dia","link":"/l/tarjeta-club-dia"},"inf_6":{"analytics_id":"globo y uber","link":"/l/facturas_3P"},"legal_1":{"analytics_id":"politica de privacidad"},"legal_2":{"analytics_id":"politica cookies","link":"/l/pol

In [43]:
respuesta_json = respuesta.json()

In [44]:
respuesta_json

{'filters_analytics': {'brand_description': {'Super Paco de Dia': {'click_parameter': 'super paco de dia',
    'filter_type': 'marca'}},
  'dia_brand': {'true': {'click_parameter': 'true'}},
  'gluten_free': {},
  'l1_category_description': {'Limpieza y hogar': {'click_parameter': 'limpieza y hogar'}},
  'l2_category_description': {'Bolsas de basura': {'click_parameter': 'bolsas de basura'}},
  'lactose_free': {},
  'nuts_free': {}},
 'footer_analytics': {'cs_1': {'analytics_id': 'atencion al cliente',
   'link': '/l/atencion-al-cliente'},
  'inf_1': {'analytics_id': 'entrega rapida'},
  'inf_2': {'analytics_id': 'envio'},
  'inf_3': {'analytics_id': 'localizador tienda',
   'link': 'https://www.dia.es/tiendas/buscador-tiendas-folletos'},
  'inf_4': {'analytics_id': 'folletos', 'link': 'https://www.dia.es/folleto'},
  'inf_5': {'analytics_id': 'tarjeta club dia', 'link': '/l/tarjeta-club-dia'},
  'inf_6': {'analytics_id': 'globo y uber', 'link': '/l/facturas_3P'},
  'legal_1': {'analyt

In [45]:
lista_productos = {}
for id in respuesta_json["page_product_analytics"].keys():
    for articulo in respuesta_json["page_product_analytics"][id]:
        lista_productos[respuesta_json["page_product_analytics"][id]["item_name"]] = respuesta_json["page_product_analytics"][id]["price"]
lista_productos

{'bolsa de basura compostable con autocierre 10 lt super paco de dia 15 unidades': 1.75,
 'bolsa de basura comunidad 120 lt super paco de dia 10 unidades': 1.55,
 'bolsa de basura con autocierre azul 30 lt super paco de dia 30 unidades': 1.59,
 'bolsa de basura con autocierre amarilla 30 lt super paco de dia 30 unidades': 1.65,
 'bolsa de basura con autocierre 50 lt super paco de dia 15 unidades': 1.6,
 'bolsa de basura con autocierre perfumada 30 lt super paco de dia 20 unidades': 1.5,
 'bolsas de basura autocierre perfumadas 10 lt super paco de dia 30 unidades': 1.01}

In [46]:
tabla = pd.DataFrame(list(lista_productos.items()), columns=['Artículo', 'Precio'])

tabla

,Artículo,Precio
0,bolsa de basura compostable con autocierre 10 ...,1.75
1,bolsa de basura comunidad 120 lt super paco de...,1.55
2,bolsa de basura con autocierre azul 30 lt supe...,1.59
3,bolsa de basura con autocierre amarilla 30 lt ...,1.65
4,bolsa de basura con autocierre 50 lt super pac...,1.60
5,bolsa de basura con autocierre perfumada 30 lt...,1.50
6,bolsas de basura autocierre perfumadas 10 lt s...,1.01


In [47]:
tabla = tabla.sort_values("Precio")
tabla 

,Artículo,Precio
6,bolsas de basura autocierre perfumadas 10 lt s...,1.01
5,bolsa de basura con autocierre perfumada 30 lt...,1.50
1,bolsa de basura comunidad 120 lt super paco de...,1.55
2,bolsa de basura con autocierre azul 30 lt supe...,1.59
4,bolsa de basura con autocierre 50 lt super pac...,1.60
3,bolsa de basura con autocierre amarilla 30 lt ...,1.65
0,bolsa de basura compostable con autocierre 10 ...,1.75


# PRODUCTO 3 - MANZANAS

In [48]:
category_name = 'frutas'
subcategory_name = 'manzanas'
category = 'L2032'
url = f"https://www.dia.es/api/v1/plp-insight/initial_analytics/{category_name}/{subcategory_name}/c/{category}?filters=categories:{category}:es&locale=es&navigation={category}&page=1&seo="

payload = {}
headers = {
  'accept': 'application/json',
  'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
  'Cookie': 'session_id=1248a3ed-bcf3-4470-a94a-0b0117288549'
}

respuesta = requests.request("GET", url, headers=headers, data=payload)
respuesta_json = respuesta.json()


In [49]:
respuesta_json

{'filters_analytics': {'caffeine_free': {},
  'dia_brand': {},
  'gluten_free': {},
  'l1_category_description': {'Frutas': {'click_parameter': 'frutas'}},
  'l2_category_description': {'Manzanas': {'click_parameter': 'manzanas'}},
  'lactose_free': {},
  'nuts_free': {},
  'salt_free': {}},
 'footer_analytics': {'cs_1': {'analytics_id': 'atencion al cliente',
   'link': '/l/atencion-al-cliente'},
  'inf_1': {'analytics_id': 'entrega rapida'},
  'inf_2': {'analytics_id': 'envio'},
  'inf_3': {'analytics_id': 'localizador tienda',
   'link': 'https://www.dia.es/tiendas/buscador-tiendas-folletos'},
  'inf_4': {'analytics_id': 'folletos', 'link': 'https://www.dia.es/folleto'},
  'inf_5': {'analytics_id': 'tarjeta club dia', 'link': '/l/tarjeta-club-dia'},
  'inf_6': {'analytics_id': 'globo y uber', 'link': '/l/facturas_3P'},
  'legal_1': {'analytics_id': 'politica de privacidad'},
  'legal_2': {'analytics_id': 'politica cookies',
   'link': '/l/politica-cookies'},
  'legal_3': {'analytics

In [50]:
lista_productos = {}
for id in respuesta_json["page_product_analytics"].keys():
    for articulo in respuesta_json["page_product_analytics"][id]:
        lista_productos[respuesta_json["page_product_analytics"][id]["item_name"]] = respuesta_json["page_product_analytics"][id]["price"]
lista_productos

{'manzana roja bolsa 1 kg': 1.99,
 'manzana golden bolsa 1 kg': 1.49,
 'manzana granny granel 800 g aprox': 1.83,
 'manzana fuji bolsa 1 kg': 2.29,
 'manzana roja granel 800 g aprox': 1.75,
 'manzana pink lady bandeja 750 gr': 2.99,
 'manzana reineta granel 1 kg aprox': 2.79,
 'manzana golden granel 500 g aprox': 1.1}

In [51]:
tabla = pd.DataFrame(list(lista_productos.items()), columns=['Artículo', 'Precio'])

tabla

,Artículo,Precio
0,manzana roja bolsa 1 kg,1.99
1,manzana golden bolsa 1 kg,1.49
2,manzana granny granel 800 g aprox,1.83
3,manzana fuji bolsa 1 kg,2.29
4,manzana roja granel 800 g aprox,1.75
5,manzana pink lady bandeja 750 gr,2.99
6,manzana reineta granel 1 kg aprox,2.79
7,manzana golden granel 500 g aprox,1.10


In [52]:
tabla = tabla.sort_values("Precio")
tabla 

,Artículo,Precio
7,manzana golden granel 500 g aprox,1.10
1,manzana golden bolsa 1 kg,1.49
4,manzana roja granel 800 g aprox,1.75
2,manzana granny granel 800 g aprox,1.83
0,manzana roja bolsa 1 kg,1.99
3,manzana fuji bolsa 1 kg,2.29
6,manzana reineta granel 1 kg aprox,2.79
5,manzana pink lady bandeja 750 gr,2.99


In [53]:
for categoria in respuesta_json["menu_analytics"].keys():
    for subcat in respuesta_json["menu_analytics"][categoria]["children"].keys():
       print(respuesta_json["menu_analytics"][categoria]["children"][subcat]["path"])

/charcuteria-y-quesos/c/L101
/charcuteria-y-quesos/jamon-cocido-lacon-fiambres-y-mortadela/c/L2001
/charcuteria-y-quesos/jamon-curado-y-paleta/c/L2004
/charcuteria-y-quesos/lomo-chorizo-fuet-salchichon/c/L2005
/charcuteria-y-quesos/queso-curado-semicurado-y-tierno/c/L2007
/charcuteria-y-quesos/queso-fresco/c/L2008
/charcuteria-y-quesos/queso-azul-y-roquefort/c/L2009
/charcuteria-y-quesos/quesos-fundidos-y-cremas/c/L2010
/charcuteria-y-quesos/quesos-internacionales/c/L2011
/charcuteria-y-quesos/foie-pate-y-sobrasada/c/L2012
/charcuteria-y-quesos/salchichas/c/L2206
/carniceria/c/L102
/carniceria/vacuno/c/L2013
/carniceria/cerdo/c/L2014
/carniceria/pavo/c/L2015
/carniceria/conejo/c/L2016
/carniceria/hamburguesas-y-carne-picada/c/L2017
/carniceria/pollo/c/L2202
/pescados-mariscos-y-ahumados/c/L103
/pescados-mariscos-y-ahumados/pescados/c/L2019
/pescados-mariscos-y-ahumados/ahumados-salazones-y-preparados/c/L2020
/pescados-mariscos-y-ahumados/sucedaneo-de-angulas-y-surimi/c/L2021
/pescados-

In [54]:
respuesta["menu_analytics"].keys()

TypeError: 'Response' object is not subscriptable

In [ ]:
response.json()["menu_analytics"]["L101"]["children"]["L2001"]["path"]

In [30]:
time.sleep(random.random()*2)